In [1]:
import hashlib
from collections import OrderedDict

from microdrop_launcher import conda_prefix
from pydash import py_ as py__
import clang_helpers as ch
import clang_helpers.clang_ast
import jinja2
import numpy as np
import pydash as py_
# Explicitly reload `clang_helpers` to pick up any changes during development.
reload(ch)
reload(clang_helpers.clang_ast)

_fp = py__()


# Create an object composed of the object properties predicate returns truthy
# for. The predicate is invoked with two arguments: (value, key).
#
# See [lodash.pickBy][1].
#
# [1]: https://lodash.com/docs/4.17.2#pickBy
py_.pick_by = lambda obj, predicate=None:\
    dict([(k, v) for k, v in obj.iteritems()
          if predicate is None or predicate(v, k)])

group_by_file = py_.pipe(py_.values,
                         _fp.group_by(lambda v: v['location']['file']))
summarize_members = _fp.map_(_fp.pick(['name', 'kind', 'type']))

In [2]:
%%writefile Test.hpp
#include <stdint.h>
#include <CArrayDefs.h>


namespace foo {
    
class Foo {
public:
  void foo(float a, uint8_t b) {}
  uint32_t foobar(uint32_t a) {
    /* Implementation of ``Foo::foobar``. */
    return a;
  }
private:
  UInt8Array _foo(UInt8Array a) { return a; }
};
    
class Bar : public Foo {
public:
  uint32_t foobar(uint32_t a) {
    /* Override ``Foo::foobar``. */
    return a * 2;
  }
};
    
}

Overwriting Test.hpp


In [3]:
include_paths = (conda_prefix().joinpath('Library', 'include'), )
                 
header_file = 'Test.hpp'
include_args = ['-I{}'.format(p) for p in include_paths]
kwargs = dict(extract_base_specifiers=True)

cpp_ast_json = ch.clang_ast.parse_cpp_ast(header_file, *include_args,
                                          format='json', **kwargs)
cpp_ast = ch.clang_ast.parse_cpp_ast(header_file, *include_args, **kwargs)

get_class = ch.clang_ast.get_class_factory(cpp_ast)
get_class_json = ch.clang_ast.get_class_factory(cpp_ast_json)

In [4]:
from IPython.lib import pretty

print 'foo::Foo'
print len('foo::Foo') * '='
pretty.pprint(get_class_json('foo::Foo'))
print ''
print 'foo::Bar'
print len('foo::Bar') * '='
pretty.pprint(get_class_json('foo::Bar'))

foo::Foo
{'base_specifiers': OrderedDict(),
 'location': {'column': 7, 'file': 'Test.hpp', 'line': 7},
 'members': OrderedDict([('foo',
               {'access_specifier': 'PUBLIC',
                'arguments': [{'const': False,
                  'kind': 'FLOAT',
                  'location': {'column': 18, 'file': 'Test.hpp', 'line': 9},
                  'name': 'a',
                  'type': 'float',
                  'underlying_type': 'float'},
                 {'const': False,
                  'kind': 'UCHAR',
                  'location': {'column': 29, 'file': 'Test.hpp', 'line': 9},
                  'name': 'b',
                  'type': 'uint8_t',
                  'underlying_type': 'unsigned char'}],
                'kind': 'FUNCTIONPROTO',
                'location': {'column': 8, 'file': 'Test.hpp', 'line': 9},
                'name': 'foo',
                'result_type': None}),
              ('foobar',
               {'access_specifier': 'PUBLIC',
                'arg

In [5]:
# Get public members of class `foo::Bar`.
public_members_i = ch.clang_ast.public_members(cpp_ast_json, 'foo::Bar')

# Function to generate a 2-byte (i.e., `uint16`) hash from a string.
f_uint16_hash = lambda v: np.fromstring(hashlib.sha1(v).digest()[:2],
                                        dtype='uint16')[0]

# Function to assign unique `uint16` code to each class and member.
f_assign_codes =  _fp.map_(py_.pipe(lambda v:
                                    py_.set_(v, 'class_code',
                                             f_uint16_hash(v['class'])),
                                    lambda v:
                                    py_.set_(v, 'member_code',
                                             f_uint16_hash(v['name']))))

# Assign unique `uint16` code to each class and member.
public_members_i = f_assign_codes(public_members_i)

# Function to group members according to type: method or attribute
f_members_by_type = _fp.group_by(lambda v: 'method'
                                 if v['kind'] == 'FUNCTIONPROTO'
                                 else 'attribute')

In [6]:
%%writefile CommandCodes.h.template
/* #########################################################################
 * [{{ now() }}] THIS FILE IS AUTOGENERATED - DO NOT EDIT!
 * ######################################################################### */
{%- for namespace_i in namespace %}
namespace {{ namespace_i }} { 
{%- endfor %}
// Map each class type to a unique numeric code.
{%- set max_len = py_.pipe(_fp.map_(_fp.get('class')), _fp.map_(py_.size), py_.max_)(public_members_i) %}
{%- for class_i in py_.uniq(py_.map_(public_members_i, _fp.pick(['class', 'class_code']))) %}
const uint16_t {{ '__'.join(class_i.class.split('::')).ljust(max_len).upper() }} = {{ class_i.class_code }};  // sha1('{{ class_i.class }}').digest()[:4]
{%- endfor %}

{% for class_i, members_i in py_.group_by(public_members_i, _fp.get('class')).iteritems() %}
{%- set members_by_type_i = f_members_by_type(members_i) %}
{% set class_namebase_i = class_i.split('::')[-1] %}
{% set namespaces_i = class_i.split('::')[:-1] %}
{%- for namespace_ij in namespaces_i %}
namespace {{ namespace_ij }} { 
{%- endfor %}
// # {{ class_i }} #
{%- if members_by_type_i.attribute %}
/* ## {{ class_i }}: attributes ##
 * 
 * Map each public **attribute** of `{{ class_i }}` to a unique numeric code.
 */
{%- set max_len = py_.size(class_namebase_i) + 2 + py_.pipe(_fp.map_(_fp.get('name')), _fp.map_(py_.size), py_.max_)(members_by_type_i.attribute) %}
{%- for member_ij in members_by_type_i.attribute %}
const uint16_t {{ class_namebase_i.upper()}}__{{ member_ij.name.ljust(max_len).upper() }} = {{ member_ij.member_code }};  // sha1('{{ member_ij.name }}').digest()[:4]
{%- endfor %}
{%- endif %}
{%- if members_by_type_i.method %}
// ## {{ class_i }}: methods ##
 * 
 * Map each public **method** of `{{ class_i }}` to a unique numeric code.
 */
{%- set max_len = py_.size(class_namebase_i) + 2 + py_.pipe(_fp.map_(_fp.get('name')), _fp.map_(py_.size), py_.max_)(members_by_type_i.method) %}
{%- for member_ij in members_by_type_i.method %}
const uint16_t {{ class_namebase_i.upper()}}__{{ member_ij.name.ljust(max_len).upper() }} = {{ member_ij.member_code }};  // sha1('{{ member_ij.name }}').digest()[:4]
{%- endfor %}
{%- endif %}
{%- for namespace_ij in namespaces_i %}
}  // namespace {{ namespace_ij }} { 
{%- endfor %}
{%- endfor %}

{%- for namespace_i in namespace %}
}  // namespace {{ namespace_i }} { 
{%- endfor %}

Overwriting CommandCodes.h.template


In [7]:
%%writefile CommandProcessor.h.template
/* #########################################################################
 * [{{ now() }}] THIS FILE IS AUTOGENERATED - DO NOT EDIT!
 * ######################################################################### */
{%- for namespace_i in namespace %}
namespace {{ namespace_i }} { 
{%- endfor %}

struct Command {
    uint16_t CLASS_CODE;
    uint32_t address;
    uint16_t MEMBER_CODE;
};

{% for class_i, members_i in py_.group_by(public_members_i, 'class').iteritems() -%}
{% set namespaces_i = class_i.split('::')[:-1] %}
{% set class_namebase_i = class_i.split('::')[-1] %}
{%- for namespace_ij in namespaces_i %}
namespace {{ namespace_ij }} { 
{%- endfor %}
struct {{ class_i.split('::')[-1] }}__CommandProcessor {
UInt8Array process_request(Command const &command, UInt8Array &message,
                           UInt8Array buffer) {
    /*
     *  If ``command.member_code`` refers to an attribute, return the
     *  corresponding attribute value of the object at ``command.address``.
     *
     *  If ``command.member_code`` refers to a method, call the corresponding
     *  method of the object at ``command.address`` and return the result.
     *  
     * Parameters
     * ----------
     * command : Command
     *     Unpacked command, including object address and member code.
     * message : UInt8Array
     *     Request message in the form:
     *     
     *         [class_code][address][member_code][arguments][payload]
     * buffer : UInt8Array
     *     Buffer to write return value to (may overlap with ``message``
     *     buffer).
     *
     * Returns
     * -------
     * UInt8Array
     *     Result of attribute access or method call, written as bytes
     *     array.
     */
    UInt8Array output = buffer;
    {{ class_i }} &obj = *reinterpret_cast<{{ class_i }} *>(command.address);
    
    switch (command.MEMBER_CODE) {
{%- set members_by_type_i = f_members_by_type(members_i) %}
{%- if members_by_type_i.attribute %}
        // # {{ class_i }}: attributes #
{%- endif %}
{%- for member_ij in members_by_type_i.attribute %}
        case {{ class_namebase_i.upper()}}__{{ member_ij.name.upper() }}:  // {{ class_i }}.{{ member_ij.name }}
        {
            {{ member_ij.type }} &result =
                *reinterpret_cast<{{ member_ij.type }} *>(output.data);
            result = obj.{{ member_ij.name }};
            output.length = sizeof({{ member_ij.type }});
            break;
        }
{%- endfor %}
{%- if members_by_type_i.method %}
        // # {{ class_i }}: methods #
{%- endif %}
{%- for member_ij in members_by_type_i.method %}
        case {{ class_namebase_i.upper()}}__{{ member_ij.name.upper() }}:  // {{ class_i }}.{{ member_ij.name }}({{ ', '.join(py_.map_(py_.zip_(py_.map_(member_ij['arguments'], _fp.get('type')), py_.map_(member_ij['arguments'], _fp.get('name'))), _fp.join(' '))) }})
        {
{%- if member_ij.result_type %}
            {{ member_ij.result_type }} &result =
                *reinterpret_cast<{{ member_ij.result_type }} *>(output.data);
            output.length = sizeof({{ member_ij.result_type }});
            result =
{%- else %}
            output.data = NULL;
            output.length = 0;
{%- endif %}
            obj.{{ member_ij.name }}({{ ', '.join(py_.map_(member_ij.arguments, _fp.get('name'))) }});
            break;
        }
{%- endfor %}
        default:
        {
            output.data = NULL;
            output.length = 0;
            break;
        }
    }
    return output;
}
};
{%- for namespace_ij in namespaces_i %}
}  // namespace {{ namespace_ij }} { 
{%- endfor %}
{% endfor %}

struct CommandProcessor {
    UInt8Array process_request(UInt8Array message, UInt8Array buffer) {
        /*
         *  1. Unpack ``Command`` from ``message`` array.
         *  2. Create a command processor for the class type corresponding to
         *     the object at the the specified ``Command.address``
         *     according to the ``Command.class_code``.
         *  3. Call the ``process_request`` method of the command processor
         *     to process the request (i.e., either return an attribute value
         *     or call a method and return the result).
         *  4. Write the bytes of the result to the ``buffer`` and update the
         *     size of the ``buffer`` based on the return type.
         *  5. Return the ``buffer``.
         *  
         * Parameters
         * ----------
         * message : UInt8Array
         *     Request message in the form:
         *     
         *         [class_code][address][member_code][arguments][payload]
         * buffer : UInt8Array
         *     Buffer to write return value to (may overlap with ``message``
         *     buffer).
         *
         * Returns
         * -------
         * UInt8Array
         *     Result of attribute access or method call, written as bytes
         *     array.
         */
        Command command = reinterpret_cast<Command *>(message.data);
{% if public_members_i %}
        switch (command.CLASS_CODE) {
{% for class_i in py_.uniq(py_.map_(public_members_i, _fp.pick(['class', 'class_code']))) %}
            case {{ '__'.join(class_i.class.split('::')).upper() }}:
            {
                // {{ class_i.class }}
                {{ class_i.class }}__CommandProcessor processor;
                processor.process_request(command, message, buffer);
            }
{% endfor %}
            default:
            {
                // NOP
                buffer.data = NULL;
                buffer.length = 0;
            }
        }
{% endif %}
        return buffer;
    }
};

{%- for namespace_i in namespace %}
}  // namespace {{ namespace_i }} { 
{%- endfor %}

Overwriting CommandProcessor.h.template


In [8]:
import datetime as dt


with open('CommandCodes.h.template', 'r') as input_:
    template = input_.read()
    
code = jinja2.Template(template).render(py_=py_, _fp=_fp,
                                        f_members_by_type=f_members_by_type,
                                        now=dt.datetime.now,
                                        public_members_i=
                                        public_members_i,
                                        namespace=['command_codes'])
print code

/* #########################################################################
 * [2016-12-06 14:39:40.318000] THIS FILE IS AUTOGENERATED - DO NOT EDIT!
 * ######################################################################### */
namespace command_codes {
// Map each class type to a unique numeric code.
const uint16_t FOO__FOO = 22740;  // sha1('foo::Foo').digest()[:4]
const uint16_t FOO__BAR = 16389;  // sha1('foo::Bar').digest()[:4]




namespace foo {
// # foo::Bar #
// ## foo::Bar: methods ##
 * 
 * Map each public **method** of `foo::Bar` to a unique numeric code.
 */
const uint16_t BAR__FOOBAR      = 17288;  // sha1('foobar').digest()[:4]
}  // namespace foo {


namespace foo {
// # foo::Foo #
// ## foo::Foo: methods ##
 * 
 * Map each public **method** of `foo::Foo` to a unique numeric code.
 */
const uint16_t FOO__FOO      = 60939;  // sha1('foo').digest()[:4]
}  // namespace foo {
}  // namespace command_codes {


In [9]:
with open('CommandProcessor.h.template', 'r') as input_:
    template = input_.read()
    
code = jinja2.Template(template).render(py_=py_, _fp=_fp,
                                        now=dt.datetime.now,
                                        f_members_by_type=f_members_by_type,
                                        public_members_i=
                                        public_members_i,
                                        namespace=['command_codes'])
print code

/* #########################################################################
 * [2016-12-06 14:39:40.374000] THIS FILE IS AUTOGENERATED - DO NOT EDIT!
 * ######################################################################### */
namespace command_codes {

struct Command {
    uint16_t CLASS_CODE;
    uint32_t address;
    uint16_t MEMBER_CODE;
};



namespace foo {
struct Bar__CommandProcessor {
UInt8Array process_request(Command const &command, UInt8Array &message,
                           UInt8Array buffer) {
    /*
     *  If ``command.member_code`` refers to an attribute, return the
     *  corresponding attribute value of the object at ``command.address``.
     *
     *  If ``command.member_code`` refers to a method, call the corresponding
     *  method of the object at ``command.address`` and return the result.
     *  
     * Parameters
     * ----------
     * command : Command
     *     Unpacked command, including object address and member code.
     * message : UInt8Arr

In [10]:
# Extract resolved element type and size from constant-sized array.
# (node.type.get_array_element_type().spelling,
#  ch.clang_ast.resolve_typedef(node.type.get_array_element_type()).kind,
#  node.type.get_array_size())

In [11]:
# Group global variable declarations by file path and summarize.
# _fp.map_values(_fp.map(_fp.pick(['name'])))(group_by_file(cpp_ast_json['members']))

In [12]:
# Get all public, non-method members of the 'dropbot_dx::Node' class.
# public_methods = f_public_methods(get_class_json('dropbot_dx::Node')
#                                   ['members'])

# py_.map_values(public_methods, _fp.pick(['typename', 'kind']))